# FASTQ download and QC

## Fetching FASTQ data from NCBI SRA

In previous weeks, we explored the genome and annotations. They were stored in FASTA and GTF files and will be used as the reference to which RNA sequencing reads are mapped/aligned. After alignment, we can count how many RNA-seq reads align to the annotations we care about (genes or transcripts).

Now, we need to get the FASTQ files that we want to align and analyze. To do so, we will be referencing an SRA read table, which is a list of all the FASTQ files associated with a given experiment uploaded to NCBI SRA. Here's the read table for the *in vivo* PZQ treatment paper:

In [ ]:
!head -11 SraRunTable.txt

You can see that there are 10 different runs, each has an ID that looks something like `SRR10776762`. After the ID, quite a bit of run metadata is provided. You should spend some time learning about what each field means.

The experiment that we're reanalyzing had 2 different samples: parasites that were harvested from untreated mice and parasites that were harvested from mice treated with 100 mg/kg PZQ. Each sample had 5 replicates, totaling 10 RNA-seq runs.

Most RNA-seq data in the USA is stored on NCBI's SRA database. We have to use `sra-tools` to download the FASTQ files from SRA. To do so, we'll use a `for` loop within bash. We can use `cut` (which we learned about in [week 2](../2_genome_exploration/2_genome_exploration.ipynb)) to get the first field, but because it's comma-delimited instead of tab-delimited, we have to use the `-d` option to tell what "delimiter" to use. The output can be saved to a file, which then can be looped through line-by-line. The FASTQ files will be stored in a new directory called `fastq`. Here's the code to get the list of runs we want to download. It will create a new file called `sra_list.txt`:

In [ ]:
!mkdir fastq
!cut -d ',' -f 1 SraRunTable.txt | tail -n +2 > sra_list.txt

Here's the code to download the FASTQ files:

In [ ]:
!while IFS= read -r line; do \
    echo "Getting $line from NCBI SRA"; \
    parallel-fastq-dump --sra-id $line --threads 16 --outdir fastq --gzip; \
    done < sra_list.txt

***11 minute time to completion***

If we take another look at the metadata, there are a few things that are of interest to us and our analysis. First, the reads are `ILLUMINA` reads generated on a `HiSeq 2500` instrument, which means they may have Illumina adapters, depending on whether or not they were trimmed by the authors prior to uploading them to SRA. Secpmd, we can see  that these are paired-end reads (`PAIRED`). This information will be important to us soon. You can get a bit more information about the data using the tool `seqkit`. For instance:

In [ ]:
!seqkit stats fastq/SRR10776762.fastq.gz

***1 minute time to completion***

## FastQC

Whenever we look at FASTQ files for the first time, we should perform quality control (QC). The primary tool used for read QC is called FastQC, which is installed in our environment. Let's take a look at the manual and then run QC on our reads:

In [ ]:
!fastqc -h

The following code uses the `*` regular expression, which will expand to any file ending in `.fastq.gz`. This line will run QC on all 10 files and use 32 cores (threads):

In [ ]:
!mkdir fastq/qc
!fastqc -t 32 fastq/*.fastq.gz -o fastq/qc

***1 minute time to completion***

Let's go through each section one-by-one.

### Per base sequence quality

Base quality scores are represented in Phred+33 encoding. Remember that Q20 is a popular threshold, as the likelihood of an error substantially increases as the Phred quality gets less than 20. It is typical to see high base quality at the 5' end of reads and lower quality at the 3' ends. For this reason, sometimes the 3' end of reads may be trimmed to remove low quality bases.

### Per tile sequence quality

Tiles are physical locations on the flow cell. Warnings in this section can be associated with flow cells that have been overloaded.

### Per sequence quality scores

This distribution shows that average Phred quality for entire reads. It helps you make decisions about removing entire reads based on their average quality.

### Per base sequence content

This distribution shows the bases most likely to be found at any given position in the reads. In general, the likelihood of finding any of A, T, G, or C at a given position should be the similar across the entire sequence. Deviations from this expectation may be due to the presence of barcodes or adaptors. Here's an example of a dataset that has the adapter on the 5' end.

<img src="assets/linker.png" alt="linker" width="450">

### Per sequence GC content

Since our reads are derived from mRNA fragments, we would expect that the percentage of Gs and Cs would be normally distributed in a way that mimics the underlying transcriptome of the organism from which the mRNA was extracted. If the distribution isn't normal (that is, if there are sharp peaks), that is an indication of some sort of contamination. Here is an example of a sharp peak:

<img src="assets/gc_peak.png" alt="GC" width="450">

This peak is indicative of something in the library that has a higher GC% than the rest of the mRNA. Sharp peaks are often associated with adapter dimers. These are reads generated from two adapters that were ligated together during library prep - they have the sequencing primer binding site, but they don't have an insert. Here's the adaptor sequence used for this library, with Gs and Cs annotated:

```
AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC = 34
 x  xxx  x xx x xx x x  x xx x x x = 18
18 / 34 = 53%
```

So, it would make sense that if many reads are associated with adaptor dimers, these reads would have a significantly higher GC% than average.

### Per base N content

Ns are called when the base caller can't make a decision for that cycle. N calls are usually associated (and correlated) with per base sequence quality.

### Sequence length distribution

Some sequencing libraries will have non-uniform read lengths, in which case we'd see a distribution of different lenghts. For runs that have a pre-defined read lenght, we should see a sharp peak at that length.

### Sequence duplication levels

For this module, FastQC will scan the first 50 bp of the first 100,000 reads and tally duplicates. The best libraries will have almost no, if any, duplicates. High levels of duplication may be indicative of PCR over amplification. Sometimes over amplification is a desired outcome; in this case, highly expressed transcripts will be highly duplicated, but this is a tradeoff for recovering more lowly expressed transcripts. FASTQs can be deduplicated before or after the alignment step.

### Overrepresented sequences

If you have many duplicate reads, it's likely there are overrepresented sequences. You can also use this list of sequences to help evalulate if you have adapter dimers. A simple way to check for problems is to search a sequence database for the overrepresented sequence. If it hits to a sequences deriving from your organism of interest, then you don't have a problem and this overrepresentation may be indicative of true underlying biology. The most useful search tool is BLAST. You can use [this website](https://blast.ncbi.nlm.nih.gov/Blast.cgi) to BLAST the sequences against the NCBI's entire nucleotide database.

### Adapter content

This module searches reads for popular adapters and PolyA stretches. If you performed mRNA enrichment, you may see higher PolyA instances at either end of the reads. 

## MultiQC

When you have many FASTQ files, all the relevant QC can be difficult to manage. MultiQC is designed to coalesce QC metrics for many different bioinformatics datatypes.

Check the manual first:

In [ ]:
!multiqc -h

MultiQC is very easy to use. All you have to do is run the command followed by the path to the working directory. It will discover all relevant QC files and wrap them up into a very helpful report. Run it now, then take a look at `multiqc_report.html` (you'll have to download it and open it in your browser).

In [ ]:
!multiqc .

# FASTQ trimming and filtering

QC metrics will inform whether or not reads should be trimmed (i.e., PolyA tails, low quality bases, or adapters removed) or filtered (low quality reads removed). Typically, manufacturers will provide some guidance on how to trim and filter sequences produced from their kit. For Illumina (the maker of the TruSeq kit used in the paper), this information can be found in the [FAQs](https://support-docs.illumina.com/SHARE/AdapterSequences/Content/SHARE/AdapterSeq/TruSeq/UDIndexes.htm). This site shows that `AGATCGGAAGAGCACACGTCTGAACTCCAGTCA` is the adapter sequence for read 1 and `AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT` is the adapter sequence for read 2. However, our FastQC eval for these data show that there are no adapters or overrepresented sequences, which means they've likely already been trimmed.

Here's what the authors of the paper said about read QC:

> Trimmed reads were mapped to the Schistosoma mansoni genome (v7.2)...

Although they likely looked at QC, they say nothing about how they trimmed or filtered.

In the end, we'll just trim low quality sequences from the end (using Q = 20 as a threshold). We will use the tool [cutadapt](https://cutadapt.readthedocs.io/en/stable/) to perform the trimming. First, check out the manual:

In [ ]:
!cutadapt --help

Edit this block and define the result of each of flags used in the above commands.

`-j`: 

`-q`: 

`-m`:

`-o`:

Now let's trim and run QC on these new files to see how they look. We'll use a bash loop again. Note that though our data was originally paired-end, the authors have already joined the read pairs so we're treating it like single-end (one read per fragment cluster).

In [ ]:
%mkdir trimmed
!for fastq in fastq/*.fastq.gz; do \
    base_name=$(basename "$fastq" .fastq.gz); \
    cutadapt -j 32 -q 20 -m 100 -o ./trimmed/$base_name.fastq.gz $fastq; \
done

***6 minute time to completion***

You can see from the cutadapt logs that very few reads were trimmed or filtered, and that's because the authors already did these steps before uploading to SRA. Not all authors do this, so we must always do QC when downloading new data to re-analyze. For our own new data, we are responsible for QC, trimming, and filtering.

Re-run FastQC so we compare the metrics before and after trimming. Generate an overall report with MultiQC. This time when running, we use the `--force` option to overwrite previous reports and the `-d` option to direct the tool to specific directories.

In [ ]:
!mkdir trimmed/qc
!fastqc -t 32 trimmed/*.fastq.gz -o trimmed/qc
!multiqc --force -d fastq/qc/ trimmed/qc/

***3 minute time to completion***

Check out the new MultiQC report to see what changed. Not a whole lot, but in some samples (i.e., SRR26691093) the PolyA tail was substantially trimmed. Our trimming did not deal with UMIs or linkers - we'll talk more about those when we get to alignment next week. You can open the individual QC files to inspect them further.

You'll notice that the "Sequence Length Distribution" sections often went from ✅ to ❌. That's because the trimmed PolyA tail was different for each read, so now we have a broad distribution of lengths rather than all of them being 69 nt. 

Even though we don't have adapters, we still have have overrepresented sequences. What are they? BLAST them against a nucleotide database and see if they are concerning or not.